In [1]:
import sys
sys.path.append("/kaggle/input/utility-smart-meter/src")

In [2]:
pip install neuralforecast torch


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Project-specific modules
from data.data_loader import load_all_raw_data
from data.data_cleaner import clean_and_merge_all_data
from features.feature_pipeline import create_comprehensive_features
from features.splitters import prepare_forecasting_data  # for train/val/test splits
from utils.helpers import reduce_memory_footprint
from utils.sequence_builder import build_global_sequences  # optional, use only if needed

# Evaluation helpers (ensure they’re model-agnostic)
from evaluation.forecast_evaluation import (
    compute_forecast_metrics,
    print_split_summary,
    evaluate_model,
    evaluate_peak_performance,
    evaluate_forecast_residuals
)

from visualization.forecast_plots import (
    plot_feature_importance,
    plot_actual_vs_predicted,
    plot_peak_actual_vs_predicted
)

from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.losses.pytorch import MSE

# Scikit-learn for generic evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Encoding (only if needed for categorical features)
from sklearn.preprocessing import LabelEncoder


In [4]:
# Set the base path where all your data files and folders are located
data_path = "/kaggle/input/smart-meters-in-london"

# Call the function to load all raw datasets: consumption, household, weather, and holiday data
raw_data = load_all_raw_data(data_path)

# Extract each dataset from the returned dictionary for easier access
df_consumption = raw_data["consumption"]  # Half-hourly electricity consumption records
df_household = raw_data["household"]      # Household metadata (e.g., tariff, ACORN group)
df_weather = raw_data["weather"]          # Daily weather data
df_holiday = raw_data["holiday"]          # UK bank holiday dates

# Display the first few rows of the consumption data to confirm successful loading
df_consumption.head()

🚀 LOADING ALL RAW DATA
📂 Found 112 consumption files
✅ Loaded 3,469,352 consumption records
📂 Loading household data...
✅ Loaded 5,566 households
📂 Loading weather data...
✅ Loaded 882 weather records
📂 Loading holiday data...
✅ Loaded 25 holiday records
🎉 ALL RAW DATA LOADED


,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
0,MAC000047,2011-12-09,0.114,0.074,0.107,0.098,0.090,0.106,0.077,0.115,...,0.097,0.134,0.295,0.063,0.072,0.040,0.074,0.051,0.065,0.068
1,MAC000047,2011-12-10,0.035,0.082,0.050,0.064,0.059,0.048,0.082,0.044,...,0.120,0.115,0.106,0.135,0.077,0.104,0.096,0.076,0.106,0.076
2,MAC000047,2011-12-11,0.103,0.085,0.083,0.113,0.072,0.096,0.098,0.084,...,0.191,0.163,0.203,0.182,0.168,0.145,0.074,0.114,0.078,0.096
3,MAC000047,2011-12-12,0.107,0.072,0.109,0.088,0.099,0.098,0.075,0.120,...,0.160,0.195,0.156,0.105,0.125,0.076,0.111,0.074,0.098,0.104
4,MAC000047,2011-12-13,0.073,0.108,0.084,0.101,0.095,0.078,0.118,0.080,...,0.181,0.130,0.146,0.116,0.113,0.110,0.082,0.120,0.079,0.101


In [5]:
# Clean and merge all raw data
df_final = clean_and_merge_all_data(raw_data)

# View the final cleaned and enriched dataset
df_final.head()

🚀 CLEANING AND MERGING ALL DATA
🧹 Cleaning consumption data...
   ✅ Removed 0 rows with >20.0% missing
   ✅ Kept 5,556 households with ≥30 days
   🔧 Interpolating missing values...
✅ Consumption data cleaned: (3469317, 50)
🏠 Preparing household data...
✅ Household data prepared: (5566, 4)
🌤️ Preparing weather data...
   ⚠️ Found 3 dates with multiple records (likely DST transitions):
   📝 Duplicate dates: [Timestamp('2014-03-30 00:00:00'), Timestamp('2013-03-31 00:00:00'), Timestamp('2012-03-25 00:00:00')]
   📊 Counts per duplicate date:
day
2014-03-30    2
2013-03-31    2
2012-03-25    2
dtype: int64
   ✅ Removed 3 DST duplicate rows
   📊 Weather data: 882 → 879 rows
✅ Weather data prepared: (879, 6)
🎉 Preparing holiday data...
✅ Holiday data prepared: (25, 3)
🔗 Merging all datasets...
   ✅ After household merge: (3469317, 53)
   ✅ After weather merge: (3469317, 58)
   ✅ After holiday merge: (3469317, 59)
✅ All data merged successfully
📅 Adding basic temporal features...
✅ Temporal fe

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,Acorn_grouped,stdorToU,temperatureMax,temperatureMin,humidity,windSpeed,cloudCover,holiday_category,month,season
0,MAC000047,2011-12-09,0.114,0.074,0.107,0.098,0.090,0.106,0.077,0.115,...,Adversity,Std,7.68,2.03,0.71,5.65,0.15,Regular Day,12,Winter
1,MAC000047,2011-12-10,0.035,0.082,0.050,0.064,0.059,0.048,0.082,0.044,...,Adversity,Std,6.08,-0.13,0.81,3.08,0.17,Regular Day,12,Winter
2,MAC000047,2011-12-11,0.103,0.085,0.083,0.113,0.072,0.096,0.098,0.084,...,Adversity,Std,8.59,2.48,0.88,3.94,0.56,Regular Day,12,Winter
3,MAC000047,2011-12-12,0.107,0.072,0.109,0.088,0.099,0.098,0.075,0.120,...,Adversity,Std,9.82,3.09,0.84,5.02,0.38,Regular Day,12,Winter
4,MAC000047,2011-12-13,0.073,0.108,0.084,0.101,0.095,0.078,0.118,0.080,...,Adversity,Std,12.08,4.54,0.75,7.44,0.42,Regular Day,12,Winter


In [6]:
# Run the full feature pipeline
df_features = create_comprehensive_features(df_final)
df_features['household_code'] = LabelEncoder().fit_transform(df_features['LCLid'])

# Check the final DataFrame
df_features.head()

print(f"✅ Feature engineering completed:")
print(f"   📊 Total samples: {len(df_features):,}")
print(f"   📅 Date range: {df_features['day'].min()} to {df_features['day'].max()}")
print(f"   🏠 Households: {df_features['LCLid'].nunique()}")
print(f"   🔧 Features created: {len(df_features.columns)} total columns")

🚀 CREATING COMPREHENSIVE FEATURES (all features retained)
📅 Creating temporal features...
🚀 Creating All Temporal Features
📅 Creating basic temporal features...
   ✅ Created basic temporal features
🌀 Creating seasonal features...
   ✅ Created seasonal features
🎉 Creating holiday features...
   ✅ Created holiday features
⚡ Creating peak period features...
   ⚠️ peak_hour column not found
📈 Creating time trend features...
   ✅ Created time trend features
✅ All temporal features created
⚡ Creating consumption‐pattern features...
⚡ Creating consumption features...
   ✅ Created consumption features
📊 Creating consumption patterns...
   ✅ Created consumption patterns
🌤️ Creating weather features...
🚀 Creating All Weather Features
🌡️ Creating temperature features...
   ✅ Created temperature features
🌦️ Creating weather condition features...
   ✅ Created weather condition features from ['humidity', 'windSpeed', 'cloudCover']
🌡️ Creating temperature impact features...
   ✅ Created temperature i

In [7]:
import gc
del df_final
gc.collect()

64

In [ ]:
#df_features = reduce_memory_footprint(df_features)

In [11]:
# Rename required columns
df_patchtst = df_features.rename(columns={
    'LCLid': 'unique_id',
    'day': 'ds',
    'total_kwh': 'y'
})

# Define selected covariates
exog_cols = [
    # Temporal context
    'dayofweek', 'month', 'is_weekend', 'is_holiday',
    'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos',
    
    # Season flags
    'is_winter', 'is_summer', 'is_shoulder_season',

    # Weather
    'temperatureMax', 'temperatureMin', 'humidity',
    'windSpeed', 'cloudCover',
    'heating_degree_days', 'cooling_degree_days',
    'is_very_cold', 'is_very_hot', 'is_high_humidity', 'is_windy',

    # Load profile
    'peak_kwh', 'mean_kwh', 'std_kwh',
    'load_factor', 'daily_variability', 'coefficient_of_variation',
    'usage_concentration', 'peak_sharpness', 'base_load', 'base_load_ratio',

    # Lag & trend
    'lag1_total', 'lag7_total', 'lag14_total',
    'roll7_total_mean', 'roll14_total_mean',
    'delta1_total', 'pct_change1_total',

    # Leakage-safe interactions
    'lag1_weekend_heating', 'lag1_holiday_consumption', 'lag1_summer_cooling'
]

# Build final dataset
df_patchtst = df_patchtst[['unique_id', 'ds', 'y'] + exog_cols].dropna()
df_patchtst = df_patchtst.sort_values(['unique_id', 'ds'])

print("✅ PatchTST-ready dataset shape:", df_patchtst.shape)
df_patchtst.head()


✅ PatchTST-ready dataset shape: (3366485, 45)


,unique_id,ds,y,dayofweek,month,is_weekend,is_holiday,month_sin,month_cos,dayofweek_sin,...,lag1_total,lag7_total,lag14_total,roll7_total_mean,roll14_total_mean,delta1_total,pct_change1_total,lag1_weekend_heating,lag1_holiday_consumption,lag1_summer_cooling
95710,MAC000002,2012-10-27,16.886,5,10,1,0,-0.866025,0.500000,-0.974928,...,15.065,17.378,11.087,16.424000,13.526500,1.937,0.147547,9.699999,0.0,0.0
95712,MAC000002,2012-10-29,12.779,0,10,0,0,-0.866025,0.500000,0.000000,...,19.629,18.885,10.257,15.659286,14.398286,2.743,0.162442,0.000000,0.0,0.0
95713,MAC000002,2012-10-30,13.961,1,10,0,0,-0.866025,0.500000,0.781831,...,12.779,10.485,9.769,14.787000,14.578429,-6.850,-0.348973,0.000000,0.0,0.0
95714,MAC000002,2012-10-31,17.822,2,10,0,0,-0.866025,0.500000,0.974928,...,13.961,15.537,10.885,15.283571,14.877857,1.182,0.092495,0.000000,0.0,0.0
95715,MAC000002,2012-11-01,12.209,3,11,0,0,-0.500000,0.866025,0.433884,...,17.822,13.128,10.751,15.610000,15.373357,3.861,0.276556,0.000000,0.0,0.0


In [15]:
# Ensure the data is sorted by ID and date
df_patchtst = df_patchtst.sort_values(['unique_id', 'ds'])

# Define cutoffs
max_date = df_patchtst['ds'].max()
val_days = 60
test_days = 60

test_start = max_date - pd.Timedelta(days=test_days)
val_start = test_start - pd.Timedelta(days=val_days)

# Split
df_train = df_patchtst[df_patchtst['ds'] < val_start]
df_val   = df_patchtst[(df_patchtst['ds'] >= val_start) & (df_patchtst['ds'] < test_start)]
df_test  = df_patchtst[df_patchtst['ds'] >= test_start]

# Summary
print(f"📅 Train: {df_train['ds'].min().date()} to {df_train['ds'].max().date()} — {len(df_train):,} rows")
print(f"📅 Val:   {df_val['ds'].min().date()} to {df_val['ds'].max().date()} — {len(df_val):,} rows")
print(f"📅 Test:  {df_test['ds'].min().date()} to {df_test['ds'].max().date()} — {len(df_test):,} rows")


📅 Train: 2011-12-08 to 2013-10-29 — 2,758,348 rows
📅 Val:   2013-10-30 to 2013-12-28 — 304,831 rows
📅 Test:  2013-12-29 to 2014-02-27 — 303,306 rows


In [13]:
import torch

In [19]:
# Define trainer arguments
trainer_args = {
    'accelerator': 'gpu' if torch.cuda.is_available() else 'cpu',
    'devices': 1,
    'strategy': 'auto'  # Safe for notebooks and multi-GPU
}

# Pass unpacked kwargs
model = PatchTST(
    h=1,
    input_size=60,
    max_steps=1000,
    scaler_type='robust',
    loss=MSE(),
    **trainer_args  # <- ✅ correct way
)

# Initialize and fit
nf = NeuralForecast(models=[model], freq='D')
nf.fit(df=df_train)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [21]:
# Predict
forecast_val = nf.predict()
forecast_val = forecast_val.rename(columns={'PatchTST': 'y_pred'})

# Merge with actuals
df_val_merged = forecast_val.merge(
    df_val[['unique_id', 'ds', 'y']], on=['unique_id', 'ds']
)

# Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

val_mae = mean_absolute_error(df_val_merged['y'], df_val_merged['y_pred'])
val_rmse = mean_squared_error(df_val_merged['y'], df_val_merged['y_pred'], squared=False)
val_r2 = r2_score(df_val_merged['y'], df_val_merged['y_pred'])
mape = np.mean(
    np.abs((df_val_merged['y'] - df_val_merged['y_pred']) / df_eval['y'].replace(0, np.nan))
) * 100
print(f"📊 Validation MAE: {val_mae:.2f}")
print(f"📊 Validation RMSE: {val_rmse:.2f}")
print(f"📈 Validation R²: {val_r2:.2f}")
print(f"🧪 Aligned Test MAPE: {mape:.2f}%")


Predicting: |          | 0/? [00:00<?, ?it/s]

📊 Validation MAE: 2.45
📊 Validation RMSE: 4.04
📈 Validation R²: 0.78
🧪 Aligned Test MAPE: 45.39%


In [22]:
# Get forecast
forecast_test = nf.predict().rename(columns={'PatchTST': 'y_pred'})

# Limit actuals to match forecast horizon
df_eval = df_patchtst.merge(forecast_test[['unique_id', 'ds']], on=['unique_id', 'ds'])
df_eval = df_eval.merge(forecast_test, on=['unique_id', 'ds'])

# Compute metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(df_eval['y'], df_eval['y_pred'])
rmse = mean_squared_error(df_eval['y'], df_eval['y_pred'], squared=False)
r2 = r2_score(df_eval['y'], df_eval['y_pred'])

mape = np.mean(
    np.abs((df_eval['y'] - df_eval['y_pred']) / df_eval['y'].replace(0, np.nan))
) * 100

print(f"🧪 Aligned Test MAE: {mae:.2f}")
print(f"🧪 Aligned Test RMSE: {rmse:.2f}")
print(f"🧪 Aligned Test R² Score: {r2:.4f}")
print(f"🧪 Aligned Test MAPE: {mape:.2f}%")


Predicting: |          | 0/? [00:00<?, ?it/s]

🧪 Aligned Test MAE: 2.45
🧪 Aligned Test RMSE: 4.04
🧪 Aligned Test R² Score: 0.7824
🧪 Aligned Test MAPE: 56.94%


In [40]:
pip install pytorch_forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-ap

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from neuralforecast.losses.pytorch import MAE
import pytorch_lightning as pl

# Set random seed for reproducibility
pl.seed_everything(42)

# Set sequence length parameters
WINDOW = 14  # Input sequence length (lookback)
H = 1        # Forecast horizon

# Assuming df_features is your processed dataframe and is already loaded
# We need to prepare it for NeuralForecast format which expects a specific structure

def prepare_data_for_neuralforecast(df, target_col='total_kwh', id_col='LCLid', time_col='day'):
    """
    Prepare smart meter data for NeuralForecast library
    
    Parameters:
    -----------
    df : pandas DataFrame
        The processed smart meter data
    target_col : str
        The target column to forecast
    id_col : str
        The household identifier column
    time_col : str
        The date column
    
    Returns:
    --------
    train_df, val_df, test_df and feature column names
    """
    # Ensure time column is datetime
    df[time_col] = pd.to_datetime(df[time_col])
    
    # Sort data
    df = df.sort_values([id_col, time_col])
    
    # Get all feature columns (excluding id, date, and target)
    exclude_cols = [id_col, time_col, target_col]
    feat_cols = [col for col in df.columns if col not in exclude_cols]
    
    # Split into train, validation, and test
    households = df[id_col].unique()
    
    # Create train/val/test splits (using the same method as in your original code)
    cutoff_dates = {}
    for hh in households:
        hh_data = df[df[id_col] == hh]
        dates = hh_data[time_col].sort_values().unique()
        
        if len(dates) < WINDOW + 60:  # Skip households with too little data
            continue
            
        val_cutoff = dates[-90-60]  # 90 test days, 60 val days
        test_cutoff = dates[-90]    # 90 test days
        
        cutoff_dates[hh] = {"val": val_cutoff, "test": test_cutoff}
    
    # Create dataframes
    train_dfs = []
    val_dfs = []
    test_dfs = []
    
    for hh in cutoff_dates:
        hh_data = df[df[id_col] == hh].copy()
        
        train_df_hh = hh_data[hh_data[time_col] <= cutoff_dates[hh]["val"]]
        val_df_hh = hh_data[(hh_data[time_col] > cutoff_dates[hh]["val"]) & 
                           (hh_data[time_col] <= cutoff_dates[hh]["test"])]
        test_df_hh = hh_data[hh_data[time_col] > cutoff_dates[hh]["test"]]
        
        train_dfs.append(train_df_hh)
        val_dfs.append(val_df_hh)
        test_dfs.append(test_df_hh)
    
    train_df = pd.concat(train_dfs)
    val_df = pd.concat(val_dfs)
    test_df = pd.concat(test_dfs)
    
    # Format specifically for NeuralForecast
    # The dataframe needs: unique_id, ds, y, and exogenous features
    train_df = train_df.rename(columns={id_col: 'unique_id', 
                                      time_col: 'ds', 
                                      target_col: 'y'})
    val_df = val_df.rename(columns={id_col: 'unique_id', 
                                   time_col: 'ds', 
                                   target_col: 'y'})
    test_df = test_df.rename(columns={id_col: 'unique_id', 
                                    time_col: 'ds', 
                                    target_col: 'y'})
    
    return train_df, val_df, test_df, feat_cols

# Example usage of the prepare function (uncomment when ready to use)
# train_df, val_df, test_df, feat_cols = prepare_data_for_neuralforecast(df_features)

# Create and train the N-BEATS model using NeuralForecast
def train_nbeats_model(train_df, val_df, feat_cols, window=14, horizon=1, max_steps=1000):
    """
    Train an N-BEATS model on smart meter data using NeuralForecast
    
    Parameters:
    -----------
    train_df : pandas DataFrame
        Training data in NeuralForecast format (unique_id, ds, y, features)
    val_df : pandas DataFrame
        Validation data in same format
    feat_cols : list
        List of feature column names
    window : int
        Input sequence length (lookback window)
    horizon : int
        Forecast horizon
    max_steps : int
        Maximum training steps
    
    Returns:
    --------
    Trained NeuralForecast model
    """
    # Configure NBEATSx model
    nbeatsx = NBEATSx(
        h=horizon,                   # Forecast horizon
        input_size=window,           # Lookback window
        futr_exog_list=feat_cols,    # Future features (known in advance)
        hist_exog_list=feat_cols,    # Historical features
        stack_types=["trend", "seasonality"],  # Use trend and seasonality stacks
        n_blocks=[3, 3],            # Number of blocks in each stack
        mlp_units=[[512, 512], [512, 512]],  # Hidden units in each block
        dropout=0.1,                 # Dropout rate
        loss=MAE(),                  # Training loss
        valid_loss=MAE(),            # Validation loss
        learning_rate=3e-4,          # Learning rate
        max_steps=max_steps,         # Maximum training steps
        
        # Trainer arguments
        accelerator="auto",          # Use GPU if available
        devices=1,                   # Number of devices
        enable_progress_bar=True,    # Show progress bar
        enable_model_summary=True,   # Show model summary
        callbacks=[pl.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss",
            patience=10,
            mode="min"
        )]
    )
    
    # Create NeuralForecast object with our model
    nf = NeuralForecast(models=[nbeatsx], freq="1D")
    
    # Calculate validation size
    val_size = len(val_df)
    
    # Train the model
    nf.fit(df=train_df, val_size=val_size)
    
    return nf

# Evaluate the model
def evaluate_nbeats_model(nf, test_df, plot_n=3):
    """
    Evaluate N-BEATS model on test data and plot results
    
    Parameters:
    -----------
    nf : NeuralForecast
        Trained NeuralForecast model
    test_df : pandas DataFrame
        Test data in NeuralForecast format
    plot_n : int
        Number of households to plot
    
    Returns:
    --------
    Evaluation metrics
    """
    # Make predictions
    forecast = nf.predict(df=test_df)
    
    # Calculate metrics
    test_df_with_preds = test_df.merge(forecast, on=['unique_id', 'ds'], how='left')
    test_df_with_preds['error'] = test_df_with_preds['y'] - test_df_with_preds['NBEATSx']
    
    # Overall metrics
    mae = test_df_with_preds['error'].abs().mean()
    mse = (test_df_with_preds['error'] ** 2).mean()
    rmse = np.sqrt(mse)
    mape = (test_df_with_preds['error'].abs() / test_df_with_preds['y'].abs()).mean() * 100
    
    print(f"Test MAE: {mae:.4f}")
    print(f"Test MSE: {mse:.4f}")
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test MAPE: {mape:.4f}%")
    
    # Plot predictions for a few households
    if plot_n > 0:
        unique_households = test_df_with_preds['unique_id'].unique()
        households_to_plot = np.random.choice(unique_households, min(plot_n, len(unique_households)), replace=False)
        
        for hh in households_to_plot:
            hh_data = test_df_with_preds[test_df_with_preds['unique_id'] == hh].sort_values('ds')
            
            plt.figure(figsize=(12, 4))
            plt.plot(hh_data['ds'], hh_data['y'], 'b-', label='Actual', marker='o')
            plt.plot(hh_data['ds'], hh_data['NBEATSx'], 'r--', label='N-BEATS Forecast', marker='x')
            plt.title(f'Household {hh}: Actual vs. Predicted Energy Consumption')
            plt.xlabel('Date')
            plt.ylabel('Energy Consumption (kWh)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
    
    return {'mae': mae, 'mse': mse, 'rmse': rmse, 'mape': mape}

# COMPLETE EXAMPLE WORKFLOW
# Uncomment and customize when ready to run
"""
# 1. Load and prepare your data
# Assuming df_features is your processed dataframe with all features
train_df, val_df, test_df, feat_cols = prepare_data_for_neuralforecast(df_features)

print(f"Training data: {train_df.shape}")
print(f"Validation data: {val_df.shape}")
print(f"Test data: {test_df.shape}")
print(f"Features: {len(feat_cols)}")

# 2. Train the N-BEATS model
nf = train_nbeats_model(
    train_df=train_df,
    val_df=val_df,
    feat_cols=feat_cols,
    window=14,
    horizon=1,
    max_steps=1000
)

# 3. Evaluate the model
metrics = evaluate_nbeats_model(nf, test_df, plot_n=3)

# 4. Save the model (optional)
import pickle
with open('nbeats_smart_meter_model.pkl', 'wb') as f:
    pickle.dump(nf, f)
"""

'\n# 1. Load and prepare your data\n# Assuming df_features is your processed dataframe with all features\ntrain_df, val_df, test_df, feat_cols = prepare_data_for_neuralforecast(df_features)\n\nprint(f"Training data: {train_df.shape}")\nprint(f"Validation data: {val_df.shape}")\nprint(f"Test data: {test_df.shape}")\nprint(f"Features: {len(feat_cols)}")\n\n# 2. Train the N-BEATS model\nnf = train_nbeats_model(\n    train_df=train_df,\n    val_df=val_df,\n    feat_cols=feat_cols,\n    window=14,\n    horizon=1,\n    max_steps=1000\n)\n\n# 3. Evaluate the model\nmetrics = evaluate_nbeats_model(nf, test_df, plot_n=3)\n\n# 4. Save the model (optional)\nimport pickle\nwith open(\'nbeats_smart_meter_model.pkl\', \'wb\') as f:\n    pickle.dump(nf, f)\n'

In [ ]:
train_df, val_df, test_df, feat_cols = prepare_data_for_neuralforecast(df_features)